#About

This notebook contains the SHiP Tracker data preparation.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Read .root file and convert the data to .csv file

In [2]:
path_to_root_file = '/Users/mikhail91/Documents/ShiP-Tracking/Data/ship.10.0.Pythia8-TGeant4.root'

Open .root file. Get list of leaves.

In [4]:
import ROOT
f=ROOT.TFile(path_to_root_file)
tree=f.Get("cbmsim")
leaves = tree.GetListOfLeaves()

leave_names = []
for i in range(0,leaves.GetEntries() ) :
    leaf = leaves.At(i)
    name = leaf.GetName()
    leave_names.append(name)

Select leaves

In [5]:
import re
f = re.compile('[a-zA-Z0-9._ ]*strawtubesPoint[a-zA-Z0-9._ ]*')
sel_leaves = []
for i in leave_names:
    res = f.findall(i)
    if res==[]:
        pass
    else:
        sel_leaves.append(res[0])

Get new columns for the .csv file

In [6]:
m = re.compile('(?<=cbmroot.strawtubes.strawtubesPoint.f)\w+')
new_cols = []
for i in sel_leaves:
    res = m.findall(i)
    if res==[]:
        pass
    else:
        new_cols.append(res[0])
new_cols = ['_'] + new_cols 

Read .root file and convert the data to the .csv file

In [8]:
import root_numpy

def set_index(list_of_leaves, index):
    new_list_of_leaves = []
    for i in range(0, len(list_of_leaves)):
        new_list_of_leaves.append( list_of_leaves[i] + '[%i]' % index )
    return new_list_of_leaves

for k in range(0, 4009):
    if k==0:
        
        data_root = root_numpy.root2array(path_to_root_file, treename='cbmsim',\
                                          branches=set_index(sel_leaves, k))
        data_slice = pd.DataFrame(data_root)
        data_slice.columns = new_cols
        data_slice['event'] = range(0, data_slice.shape[0])
        data_slice['k'] = k*np.ones(data_slice.shape[0])
        data_slice = data_slice[data_slice.DetectorID > 0]
        data_full = data_slice
    else:
        data_root = root_numpy.root2array(path_to_root_file, treename='cbmsim',\
                                          branches=set_index(sel_leaves, k))
        data_slice = pd.DataFrame(data_root)
        data_slice.columns = new_cols
        data_slice['event'] = range(0, data_slice.shape[0])
        data_slice['k'] = k*np.ones(data_slice.shape[0])
        data_slice = data_slice[data_slice.DetectorID > 0]
        data_full = pd.concat([data_full, data_slice], axis=0)

In [10]:
data_full.sort(['event', 'k'], inplace=True)
data_full.reset_index(inplace=True)

In [11]:
data_full.irow(range(0,10))

,index,_,UniqueID,Bits,RefToLinks,TrackID,EventId,Px,Py,Pz,...,Length,ELoss,DetectorID,X,Y,Z,PdgCode,dist2Wire,event,k
0,0,0,0,50331648,0,3,0,-0.358307,-0.329753,15.881699,...,0.139202,0.000000,50002319,8.883409,-61.202305,-1974.859985,-13,0.274822,0,0
1,0,0,0,50331648,0,3,0,-0.358236,-0.330276,15.881608,...,0.623953,0.000002,50012320,8.858191,-61.225544,-1973.742188,-13,0.582455,0,1
2,0,0,0,50331648,0,3,0,-0.358310,-0.330085,15.881518,...,0.955728,0.000044,50102319,8.824563,-61.256523,-1972.251953,-13,0.111252,0,2
3,0,0,0,50331648,0,3,0,-0.358471,-0.329928,15.881445,...,0.675162,0.000003,51012320,8.632817,-61.432972,-1963.756592,-13,0.164412,0,3
4,0,0,0,50331648,0,3,0,-0.358435,-0.329922,15.881428,...,0.971867,0.000005,51112320,8.573978,-61.487133,-1961.149292,-13,0.226827,0,4
5,0,0,0,50331648,0,3,0,-0.358559,-0.306676,15.881873,...,0.798835,0.000001,10002373,-93.959763,-155.480576,2581.155273,-13,0.279474,0,5
6,0,0,0,50331648,0,3,0,-0.358723,-0.306194,15.881861,...,0.857819,0.000005,10112373,-94.043098,-155.551666,2584.845459,-13,0.231709,0,6
7,0,0,0,50331648,0,3,0,-0.358613,-0.305509,15.881857,...,0.958992,0.000004,11002368,-94.185486,-155.673019,2591.151611,-13,0.088185,0,7
8,0,0,0,50331648,0,3,0,-0.358743,-0.306154,15.881789,...,0.122994,0.000000,11102368,-94.244331,-155.723068,2593.758057,-13,0.483507,0,8
9,0,0,0,50331648,0,3,0,-0.358271,-0.306372,15.881742,...,0.510439,0.000003,11112368,-94.268814,-155.743942,2594.842773,-13,0.415283,0,9


In [13]:
#data_full.to_csv('data/strawtubesPoint.csv')

#Read Data

Open .csv file with hits.

In [14]:
path_to_data = 'data/strawtubesPoint.csv'

In [16]:
data = pd.read_csv(path_to_data, delimiter=',')
data.irow(range(0,10))

,Unnamed: 0,index,_,UniqueID,Bits,RefToLinks,TrackID,EventId,Px,Py,...,Length,ELoss,DetectorID,X,Y,Z,PdgCode,dist2Wire,event,k
0,0,0,0,0,50331648,0,3,0,-0.358307,-0.329753,...,0.139202,0.000000,50002319,8.883409,-61.202305,-1974.859985,-13,0.274822,0,0
1,1,0,0,0,50331648,0,3,0,-0.358236,-0.330276,...,0.623953,0.000002,50012320,8.858191,-61.225544,-1973.742188,-13,0.582455,0,1
2,2,0,0,0,50331648,0,3,0,-0.358310,-0.330085,...,0.955728,0.000044,50102319,8.824563,-61.256523,-1972.251953,-13,0.111252,0,2
3,3,0,0,0,50331648,0,3,0,-0.358471,-0.329928,...,0.675162,0.000003,51012320,8.632817,-61.432972,-1963.756592,-13,0.164412,0,3
4,4,0,0,0,50331648,0,3,0,-0.358435,-0.329922,...,0.971867,0.000005,51112320,8.573978,-61.487133,-1961.149292,-13,0.226827,0,4
5,5,0,0,0,50331648,0,3,0,-0.358559,-0.306676,...,0.798835,0.000001,10002373,-93.959763,-155.480576,2581.155273,-13,0.279474,0,5
6,6,0,0,0,50331648,0,3,0,-0.358723,-0.306194,...,0.857819,0.000005,10112373,-94.043098,-155.551666,2584.845459,-13,0.231709,0,6
7,7,0,0,0,50331648,0,3,0,-0.358613,-0.305509,...,0.958992,0.000004,11002368,-94.185486,-155.673019,2591.151611,-13,0.088185,0,7
8,8,0,0,0,50331648,0,3,0,-0.358743,-0.306154,...,0.122994,0.000000,11102368,-94.244331,-155.723068,2593.758057,-13,0.483507,0,8
9,9,0,0,0,50331648,0,3,0,-0.358271,-0.306372,...,0.510439,0.000003,11112368,-94.268814,-155.743942,2594.842773,-13,0.415283,0,9


Drop not informative columns

In [19]:
data.drop(['_', 'RefToLinks', 'index','UniqueID', 'EventId'], axis=1, inplace=True)
data.irow(range(0,10))

,Unnamed: 0,Bits,TrackID,Px,Py,Pz,Time,Length,ELoss,DetectorID,X,Y,Z,PdgCode,dist2Wire,event,k
0,0,50331648,3,-0.358307,-0.329753,15.881699,232.044861,0.139202,0.000000,50002319,8.883409,-61.202305,-1974.859985,-13,0.274822,0,0
1,1,50331648,3,-0.358236,-0.330276,15.881608,232.074081,0.623953,0.000002,50012320,8.858191,-61.225544,-1973.742188,-13,0.582455,0,1
2,2,50331648,3,-0.358310,-0.330085,15.881518,232.118286,0.955728,0.000044,50102319,8.824563,-61.256523,-1972.251953,-13,0.111252,0,2
3,3,50331648,3,-0.358471,-0.329928,15.881445,232.406479,0.675162,0.000003,51012320,8.632817,-61.432972,-1963.756592,-13,0.164412,0,3
4,4,50331648,3,-0.358435,-0.329922,15.881428,232.488541,0.971867,0.000005,51112320,8.573978,-61.487133,-1961.149292,-13,0.226827,0,4
5,5,50331648,3,-0.358559,-0.306676,15.881873,384.080780,0.798835,0.000001,10002373,-93.959763,-155.480576,2581.155273,-13,0.279474,0,5
6,6,50331648,3,-0.358723,-0.306194,15.881861,384.202942,0.857819,0.000005,10112373,-94.043098,-155.551666,2584.845459,-13,0.231709,0,6
7,7,50331648,3,-0.358613,-0.305509,15.881857,384.411713,0.958992,0.000004,11002368,-94.185486,-155.673019,2591.151611,-13,0.088185,0,7
8,8,50331648,3,-0.358743,-0.306154,15.881789,384.512634,0.122994,0.000000,11102368,-94.244331,-155.723068,2593.758057,-13,0.483507,0,8
9,9,50331648,3,-0.358271,-0.306372,15.881742,384.542358,0.510439,0.000003,11112368,-94.268814,-155.743942,2594.842773,-13,0.415283,0,9


#The detectorID decoding.

The station number (statnb = 1,2,3,4; for the Veto station statnb = 5),

The viewnumber (vnb = 0,1,2,3),

The planenumber (pnb = 0,1),

The layernumber (lnb = 0,1),

The strawnumber (snb)

In [20]:
detID = data.DetectorID.values.astype(np.int32)
StatNb = []
ViewNb = []
PlaneNb = []
LayerNb = []
StrawNb = []
WireX = []
WireY = []
WireZ = []
for i in detID:
    statnb = i//10000000
    vnb = (i - statnb*10000000)//1000000
    pnb = (i - statnb*10000000 - vnb*1000000)//100000
    lnb = (i - statnb*10000000 - vnb*1000000 - pnb*100000)//10000
    snb = i - statnb*10000000 - vnb*1000000 - pnb*100000 - lnb*10000 - 2000
    
    StatNb.append(statnb)
    ViewNb.append(vnb)
    PlaneNb.append(pnb)
    LayerNb.append(lnb)
    StrawNb.append(snb)
    
data['StatNb'] = StatNb
data['ViewNb'] = ViewNb
data['PlaneNb'] = PlaneNb
data['LayerNb'] = LayerNb
data['StrawNb'] = StrawNb

In [21]:
data.irow(range(0,10))

,Unnamed: 0,Bits,TrackID,Px,Py,Pz,Time,Length,ELoss,DetectorID,...,Z,PdgCode,dist2Wire,event,k,StatNb,ViewNb,PlaneNb,LayerNb,StrawNb
0,0,50331648,3,-0.358307,-0.329753,15.881699,232.044861,0.139202,0.000000,50002319,...,-1974.859985,-13,0.274822,0,0,5,0,0,0,319
1,1,50331648,3,-0.358236,-0.330276,15.881608,232.074081,0.623953,0.000002,50012320,...,-1973.742188,-13,0.582455,0,1,5,0,0,1,320
2,2,50331648,3,-0.358310,-0.330085,15.881518,232.118286,0.955728,0.000044,50102319,...,-1972.251953,-13,0.111252,0,2,5,0,1,0,319
3,3,50331648,3,-0.358471,-0.329928,15.881445,232.406479,0.675162,0.000003,51012320,...,-1963.756592,-13,0.164412,0,3,5,1,0,1,320
4,4,50331648,3,-0.358435,-0.329922,15.881428,232.488541,0.971867,0.000005,51112320,...,-1961.149292,-13,0.226827,0,4,5,1,1,1,320
5,5,50331648,3,-0.358559,-0.306676,15.881873,384.080780,0.798835,0.000001,10002373,...,2581.155273,-13,0.279474,0,5,1,0,0,0,373
6,6,50331648,3,-0.358723,-0.306194,15.881861,384.202942,0.857819,0.000005,10112373,...,2584.845459,-13,0.231709,0,6,1,0,1,1,373
7,7,50331648,3,-0.358613,-0.305509,15.881857,384.411713,0.958992,0.000004,11002368,...,2591.151611,-13,0.088185,0,7,1,1,0,0,368
8,8,50331648,3,-0.358743,-0.306154,15.881789,384.512634,0.122994,0.000000,11102368,...,2593.758057,-13,0.483507,0,8,1,1,1,0,368
9,9,50331648,3,-0.358271,-0.306372,15.881742,384.542358,0.510439,0.000003,11112368,...,2594.842773,-13,0.415283,0,9,1,1,1,1,368


Write the data.

In [23]:
#data.to_csv('data/strawtubesPoint_decoded_hits.csv')